# **Import the data from PokeAPI**

In [1]:
import pandas as pd
import requests

uri = "https://pokeapi.co/api/v2/pokemon/?limit=10"
response = requests.get(uri)

results = response.json()["results"]

pokemons = pd.DataFrame(results)

pokemons.head()

,name,url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/
3,charmander,https://pokeapi.co/api/v2/pokemon/4/
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/


# **Generates a dictionary with the Pokemon information**

In [2]:
from pprint import pprint

pokemon_info = []
irrelevant_data = ['moves', 'stats', 'game_indices',
                   'abilities', 'held_items', 'forms',
                   'base_experience', 'is_default',
                   'order', 'id', 'sprites',
                   'location_area_encounters']

for pokemon in pokemons.iterrows():
  index = pokemon[0] + 1
  uri = f'https://pokeapi.co/api/v2/pokemon/{index}'
  
  response = requests.get(uri)

  info = response.json()
  info["index"] = index

  name = pokemon[1]["name"]
  info["name"] = name

  sprite = info["sprites"]["front_default"]
  info["sprite"] = sprite

  types = list(map(lambda t: t["type"]["name"], info["types"]))
  info["types"] = types

  # Removes irrelevant data
  for data in irrelevant_data:
    info.pop(data)

  pokemon_info.append(info)

pprint(pokemon_info[0])

{'height': 7,
 'index': 1,
 'name': 'bulbasaur',
 'species': {'name': 'bulbasaur',
             'url': 'https://pokeapi.co/api/v2/pokemon-species/1/'},
 'sprite': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png',
 'types': ['poison', 'grass'],
 'weight': 69}


# **Get information from other end points**

In [3]:
for pokemon in pokemon_info:
  uri = pokemon["species"]["url"]

  response = requests.get(uri)
  info = response.json()

  for flavor_text in info["flavor_text_entries"]:
    if flavor_text["language"]["name"] == "en":
      pokemon["description"] = flavor_text["flavor_text"]
      break

  for genus in info["genera"]:
    if genus["language"]["name"] == "en":
      pokemon["specie"] = genus["genus"]
      break
  
  pokemon.pop("species")


pprint(pokemon_info[9])

  

{'description': 'Perhaps because it would like to grow up\n'
                'quickly, it has a voracious appetite, eating\n'
                'a hundred leaves a day.',
 'height': 3,
 'index': 10,
 'name': 'caterpie',
 'specie': 'Worm Pokémon',
 'sprite': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/10.png',
 'types': ['bug'],
 'weight': 29}
